### Importações

In [1]:
import re
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import *

### Carregando os Dados

In [2]:
texto = open('texto.txt', 'r').read()

In [3]:
type(texto)

str

In [4]:
print(texto)

'Olá, como vai? Eu sou a Camila.\n'
'Olá, Camila, meu nome é Fernando. Muito prazer.\n'
'Prazer em conhecer você também. Como você está hoje?\n'
'Ótimo. Meu time de futebol venceu a competição.\n'
'Uau, Parabéns Fernando!\n'
'Obrigado Camila.\n'
'Vamos comer uma pizza mais tarde para celebrar?\n'
'Claro. Você recomenda algum restaurante Camila?\n'
'Sim, abriu um restaurante novo e dizem que a pizza de banana é fenomenal.\n'
'Ok. Nos encontramos no restaurante às sete da noite, pode ser?\n'
'Pode sim. Nos vemos mais tarde então.'


### Pré-Processamento dos Dados e Construção do Vocabulário

In [5]:
sentences = re.sub("[.,!?\\-]", '', texto.lower()).split('\n')

In [6]:
print(sentences)

["'olá como vai eu sou a camila\\n'", "'olá camila meu nome é fernando muito prazer\\n'", "'prazer em conhecer você também como você está hoje\\n'", "'ótimo meu time de futebol venceu a competição\\n'", "'uau parabéns fernando\\n'", "'obrigado camila\\n'", "'vamos comer uma pizza mais tarde para celebrar\\n'", "'claro você recomenda algum restaurante camila\\n'", "'sim abriu um restaurante novo e dizem que a pizza de banana é fenomenal\\n'", "'ok nos encontramos no restaurante às sete da noite pode ser\\n'", "'pode sim nos vemos mais tarde então'"]


In [7]:
word_list = list(set(" ".join(sentences).split()))

In [8]:
print(word_list)

["competição\\n'", 'um', 'parabéns', "'claro", 'banana', 'a', "ser\\n'", 'venceu', 'restaurante', 'pode', "'sim", "'pode", "então'", 'uma', "'ok", 'no', 'nos', 'abriu', 'conhecer', "fenomenal\\n'", "'uau", 'em', 'sete', 'e', 'camila', 'tarde', 'eu', "hoje\\n'", 'encontramos', 'vemos', "'olá", 'mais', 'vai', 'como', 'nome', 'futebol', "'obrigado", 'time', "celebrar\\n'", 'algum', "'prazer", 'dizem', 'que', 'é', 'você', 'sou', 'de', 'recomenda', 'às', 'noite', "prazer\\n'", 'fernando', 'muito', "fernando\\n'", "'vamos", 'novo', 'sim', "camila\\n'", 'também', 'meu', 'comer', 'para', 'da', "'ótimo", 'está', 'pizza']


In [9]:
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

In [10]:
print(word_dict)

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}


In [11]:
for i, w in enumerate(word_list):
    word_dict[w] = i + 4

In [12]:
print(word_dict)

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3, "competição\\n'": 4, 'um': 5, 'parabéns': 6, "'claro": 7, 'banana': 8, 'a': 9, "ser\\n'": 10, 'venceu': 11, 'restaurante': 12, 'pode': 13, "'sim": 14, "'pode": 15, "então'": 16, 'uma': 17, "'ok": 18, 'no': 19, 'nos': 20, 'abriu': 21, 'conhecer': 22, "fenomenal\\n'": 23, "'uau": 24, 'em': 25, 'sete': 26, 'e': 27, 'camila': 28, 'tarde': 29, 'eu': 30, "hoje\\n'": 31, 'encontramos': 32, 'vemos': 33, "'olá": 34, 'mais': 35, 'vai': 36, 'como': 37, 'nome': 38, 'futebol': 39, "'obrigado": 40, 'time': 41, "celebrar\\n'": 42, 'algum': 43, "'prazer": 44, 'dizem': 45, 'que': 46, 'é': 47, 'você': 48, 'sou': 49, 'de': 50, 'recomenda': 51, 'às': 52, 'noite': 53, "prazer\\n'": 54, 'fernando': 55, 'muito': 56, "fernando\\n'": 57, "'vamos": 58, 'novo': 59, 'sim': 60, "camila\\n'": 61, 'também': 62, 'meu': 63, 'comer': 64, 'para': 65, 'da': 66, "'ótimo": 67, 'está': 68, 'pizza': 69}


In [13]:
number_dict = {i: w for i, w in enumerate(word_dict)}

In [14]:
number_dict

{0: '[PAD]',
 1: '[CLS]',
 2: '[SEP]',
 3: '[MASK]',
 4: "competição\\n'",
 5: 'um',
 6: 'parabéns',
 7: "'claro",
 8: 'banana',
 9: 'a',
 10: "ser\\n'",
 11: 'venceu',
 12: 'restaurante',
 13: 'pode',
 14: "'sim",
 15: "'pode",
 16: "então'",
 17: 'uma',
 18: "'ok",
 19: 'no',
 20: 'nos',
 21: 'abriu',
 22: 'conhecer',
 23: "fenomenal\\n'",
 24: "'uau",
 25: 'em',
 26: 'sete',
 27: 'e',
 28: 'camila',
 29: 'tarde',
 30: 'eu',
 31: "hoje\\n'",
 32: 'encontramos',
 33: 'vemos',
 34: "'olá",
 35: 'mais',
 36: 'vai',
 37: 'como',
 38: 'nome',
 39: 'futebol',
 40: "'obrigado",
 41: 'time',
 42: "celebrar\\n'",
 43: 'algum',
 44: "'prazer",
 45: 'dizem',
 46: 'que',
 47: 'é',
 48: 'você',
 49: 'sou',
 50: 'de',
 51: 'recomenda',
 52: 'às',
 53: 'noite',
 54: "prazer\\n'",
 55: 'fernando',
 56: 'muito',
 57: "fernando\\n'",
 58: "'vamos",
 59: 'novo',
 60: 'sim',
 61: "camila\\n'",
 62: 'também',
 63: 'meu',
 64: 'comer',
 65: 'para',
 66: 'da',
 67: "'ótimo",
 68: 'está',
 69: 'pizza'}

In [15]:
vocab_size = len(word_dict)
print(vocab_size)

70


In [16]:
token_list = list()

In [17]:
for sentence in sentences:
    arr = [word_dict[s] for s in sentence.split()]
    token_list.append(arr)

In [18]:
token_list

[[34, 37, 36, 30, 49, 9, 61],
 [34, 28, 63, 38, 47, 55, 56, 54],
 [44, 25, 22, 48, 62, 37, 48, 68, 31],
 [67, 63, 41, 50, 39, 11, 9, 4],
 [24, 6, 57],
 [40, 61],
 [58, 64, 17, 69, 35, 29, 65, 42],
 [7, 48, 51, 43, 12, 61],
 [14, 21, 5, 12, 59, 27, 45, 46, 9, 69, 50, 8, 47, 23],
 [18, 20, 32, 19, 12, 52, 26, 66, 53, 13, 10],
 [15, 60, 20, 33, 35, 29, 16]]

In [19]:
texto[0:29]

"'Olá, como vai? Eu sou a Cami"

In [20]:
token_list[0]

[34, 37, 36, 30, 49, 9, 61]

### Definição dos Hiperparâmetros

In [21]:
# Hiperparâmetros
batch_size = 6
n_segments = 2
dropout = 0.2

# Comprimento máximo
maxlen = 100

# Número máximo de tokens que serão previstos
max_pred = 7

# Número de Camadas
n_layers = 6

# Número de cabeças no multi-head attention
n_heads = 12

# Tamanho da embedding
d_model = 768

# Tamanho da dimensão feedforward: 4 * d_model
d_ff = d_model * 4

# Dimensão de K(=Q)V
d_k = d_v = 64

# Epochs
NUM_EPOCHS = 50

### Criação dos Batches de Dados e Aplicação dos Tokens Especiais

In [22]:
def make_batch():

    batch = []
    positive = negative = 0
    while positive != batch_size/2 or negative != batch_size/2:
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]
        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)
        n_pred = min(max_pred, max(1, int(round(len(input_ids) * 0.15))))
        cand_maked_pos = [i for i, token in enumerate(input_ids) if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])

            if random() < 0.8:
                input_ids[pos] = word_dict['[MASK]']

            elif random() < 0.5:
                index = randint(0, vocab_size - 1)
                input_ids[pos] = word_dict[number_dict[index]]

        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True])
            positive += 1

        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False])
            negative += 1

    return batch

In [23]:
def get_attn_pad_masked(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_masked = seq_k.data.eq(0).unsqueeze(1)
    return pad_attn_masked.expand(batch_size, len_q, len_k)

In [24]:
batch = make_batch()

In [25]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

In [26]:
input_ids

tensor([[ 1,  3, 64, 17, 69, 35, 29, 65, 42,  2, 10, 20, 32, 19, 12, 52, 26, 66,
         53,  3, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 67, 63, 41, 50, 39,  3,  9,  4,  2,  3, 63, 41, 50, 39,  3,  9,  4,
          2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 34, 37, 36,  3, 49,  9, 61,  2, 67, 63, 41, 50,  3, 11,  9,  3,  2,
         

In [27]:
input_ids[0]

tensor([ 1,  3, 64, 17, 69, 35, 29, 65, 42,  2, 10, 20, 32, 19, 12, 52, 26, 66,
        53,  3, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [28]:
segment_ids[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [29]:
masked_tokens[0]

tensor([58, 18, 13,  0,  0,  0,  0])

In [30]:
masked_pos[0]

tensor([ 1, 10, 19,  0,  0,  0,  0])

In [31]:
isNext[0]

tensor(0)

In [32]:
get_attn_pad_masked(input_ids, input_ids)[0][0], input_ids[0]

(tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True]),
 tensor([ 1,  3, 64, 17, 69, 35, 29, 65, 42,  2, 10, 20, 32, 19, 12, 52, 26, 66,
         53,  3, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  

### Construção do Modelo

#### Função de Ativação

In [33]:
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

#### Módulo Embedding

In [34]:
class Embedding(nn.Module):

    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(maxlen, d_model)
        self.seg_embed = nn.Embedding(n_segments, d_model)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype = torch.long)
        pos = pos.unsqueeze(0).expand_as(x)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)

        return self.norm(embedding)

#### Módulo Scaled Dot Product Attention

In [35]:
class ScaledDotProductAttention(nn.Module):

    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)
        scores.masked_fill_(attn_mask, -1e9)
        attn = nn.Softmax(dim = -1)(scores)
        context = torch.matmul(attn, V)

        return context, attn

#### Módulo Multi-Head Attention

In [36]:
class MultiHeadAttention(nn.Module):

    def __init__(self) -> None:
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_k * n_heads)

    def forward(self, Q, K, V, attn_mask):
        residual, batch_size = Q, Q.size(0)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v)
        output = nn.Linear(n_heads * d_v, d_model)(context)

        return nn.LayerNorm(d_model)(output + residual), attn

In [37]:
emb = Embedding()

In [38]:
embeds = emb(input_ids, segment_ids)

In [39]:
attenM = get_attn_pad_masked(input_ids, input_ids)

In [40]:
MHA = MultiHeadAttention()(embeds, embeds, embeds, attenM)

In [41]:
output, A = MHA

In [42]:
A[0][0]

tensor([[0.0436, 0.0261, 0.0357,  ..., 0.0000, 0.0000, 0.0000],
        [0.0485, 0.0176, 0.0372,  ..., 0.0000, 0.0000, 0.0000],
        [0.0768, 0.0598, 0.0226,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0884, 0.0363, 0.0379,  ..., 0.0000, 0.0000, 0.0000],
        [0.0556, 0.0407, 0.0432,  ..., 0.0000, 0.0000, 0.0000],
        [0.0559, 0.0396, 0.0465,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<SelectBackward0>)

#### Módulo Feedforward Posicional

In [54]:
class PoswiseFeedForward(nn.Module):

    def __init__(self) -> None:
        super(PoswiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.fc2(gelu(self.fc1(x)))

#### Módulo Encoder Layer

In [55]:
class EncoderLayer(nn.Module):

    def __init__(self) -> None:
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForward()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_inputs, atnn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)
        enc_inputs = self.pos_ffn(enc_inputs)

        return enc_inputs, atnn

#### Arquitetura Final do LLM (Modelo BERT)

In [56]:
class BERT(nn.Module):

    def __init__(self) -> None:
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias = False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_masked(input_ids, input_ids)

        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)

        h_pooled = self.activ1(self.fc(output[:, 0]))
        logits_clsf = self.classifier(h_pooled)
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1))
        h_masked = torch.gather(output, 1, masked_pos)
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias

        return logits_lm, logits_clsf

### Treinamento e Avaliação do LLM

In [57]:
modelo = BERT()

In [58]:
criterion = nn.CrossEntropyLoss()

In [59]:
optimizer = optim.Adam(modelo.parameters(), lr = 0.001)

In [60]:
batch = make_batch()

In [61]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

In [62]:
%%time

for epoch in range(NUM_EPOCHS):
    optimizer.zero_grad()
    logits_lm, logits_clsf = modelo(input_ids, segment_ids, masked_pos)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens)
    loss_lm = (loss_lm.float()).mean()
    loss_clsf = criterion(logits_clsf, isNext)
    loss = loss_lm + loss_clsf
    print(f'Epoch: {epoch + 1} | Loss: {loss:.4f}')
    loss.backward()
    optimizer.step()

Epoch: 1 | Loss: 59.0222
Epoch: 2 | Loss: 79.8279
Epoch: 3 | Loss: 304.2738
Epoch: 4 | Loss: 158.2007
Epoch: 5 | Loss: 42.0082
Epoch: 6 | Loss: 50.1120
Epoch: 7 | Loss: 19.0199
Epoch: 8 | Loss: 30.2061
Epoch: 9 | Loss: 30.8063
Epoch: 10 | Loss: 26.3319
Epoch: 11 | Loss: 22.5527
Epoch: 12 | Loss: 23.5192
Epoch: 13 | Loss: 23.1593
Epoch: 14 | Loss: 21.2182
Epoch: 15 | Loss: 21.6606
Epoch: 16 | Loss: 18.4302
Epoch: 17 | Loss: 26.8563
Epoch: 18 | Loss: 26.0158
Epoch: 19 | Loss: 22.7878
Epoch: 20 | Loss: 18.7549
Epoch: 21 | Loss: 21.5214
Epoch: 22 | Loss: 21.7321
Epoch: 23 | Loss: 20.1513
Epoch: 24 | Loss: 19.6701
Epoch: 25 | Loss: 30.5417
Epoch: 26 | Loss: 21.2938
Epoch: 27 | Loss: 17.8672
Epoch: 28 | Loss: 17.8167
Epoch: 29 | Loss: 17.5997
Epoch: 30 | Loss: 16.9462
Epoch: 31 | Loss: 15.2093
Epoch: 32 | Loss: 14.2231
Epoch: 33 | Loss: 15.6887
Epoch: 34 | Loss: 13.8837
Epoch: 35 | Loss: 13.5548
Epoch: 36 | Loss: 12.8815
Epoch: 37 | Loss: 14.0424
Epoch: 38 | Loss: 14.1079
Epoch: 39 | Loss: 1

### Extraindo as Previsões do LLM Treinado

In [63]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
print(texto)
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

'Olá, como vai? Eu sou a Camila.\n'
'Olá, Camila, meu nome é Fernando. Muito prazer.\n'
'Prazer em conhecer você também. Como você está hoje?\n'
'Ótimo. Meu time de futebol venceu a competição.\n'
'Uau, Parabéns Fernando!\n'
'Obrigado Camila.\n'
'Vamos comer uma pizza mais tarde para celebrar?\n'
'Claro. Você recomenda algum restaurante Camila?\n'
'Sim, abriu um restaurante novo e dizem que a pizza de banana é fenomenal.\n'
'Ok. Nos encontramos no restaurante às sete da noite, pode ser?\n'
'Pode sim. Nos vemos mais tarde então.'
['[CLS]', "'ótimo", 'meu', 'time', "ser\\n'", 'futebol', 'venceu', 'a', "competição\\n'", '[SEP]', "'olá", '[MASK]', 'vai', 'eu', 'sou', 'a', "camila\\n'", '[SEP]']


In [64]:
logits_lm, logits_clsf = modelo(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print(f'Lista de Masked Tokens Reais: {[pos.item() for pos in masked_tokens[0] if pos.item() != 0]}')
print(f'Lista de Masked Tokens Previstos: {[pos for pos in logits_lm if pos != 0]}')

Lista de Masked Tokens Reais: [37, 11, 50]
Lista de Masked Tokens Previstos: [35, 35, 35]


In [65]:
logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print(f'isNext (Valor Real): {True if isNext else False}')
print(f'isNext (Valor Previsto): {True if logits_clsf else False}')

isNext (Valor Real): False
isNext (Valor Previsto): True
